##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Clasificación de texto con Model Maker de TensorFlow Lite

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/lite/models/modify/model_maker/text_classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/lite/models/modify/model_maker/text_classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/lite/models/modify/model_maker/text_classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/lite/models/modify/model_maker/text_classification.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
</table>

La biblioteca [Model Maker de TensorFlow Lite](https://www.tensorflow.org/lite/models/modify/model_maker) simplifica el proceso de adaptación y conversión de un modelo TensorFlow a unos datos de entrada concretos cuando se implementa este modelo para aplicaciones de ML en dispositivos.

Este bloc muestra un ejemplo de principio a fin que utiliza la biblioteca Model Maker para ilustrar la adaptación y conversión de un modelo de clasificación de texto de uso común para clasificar reseñas de películas en un dispositivo móvil. El modelo de clasificación de texto clasifica el texto en categorías predefinidas. Las entradas deben ser texto preprocesado y las salidas son las probabilidades de las categorías. El conjunto de datos usado en este tutorial son críticas de películas positivas y negativas.

## Requisitos previos


### Instalar los paquetes necesarios

Para ejecutar este ejemplo, instale los paquetes necesarios, incluido el paquete Model Maker del repositorio [GitHub](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker).

In [ ]:
!sudo apt -y install libportaudio2
!pip install -q tflite-model-maker
!pip uninstall tflite_support_nightly
!pip install tflite_support_nightly

Importe los paquetes necesarios.

In [ ]:
import numpy as np
import os

from tflite_model_maker import model_spec
from tflite_model_maker import text_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.text_classifier import AverageWordVecSpec
from tflite_model_maker.text_classifier import DataLoader

from tflite_support.task import core
from tflite_support.task import processor
from tflite_support.task import text

import tensorflow as tf
assert tf.__version__.startswith('2')
tf.get_logger().setLevel('ERROR')

### Descargar los datos de entrenamiento de muestra.

En este tutorial, usaremos el [SST-2](https://nlp.stanford.edu/sentiment/index.html) (Stanford Sentiment Treebank), que es una de las tareas del banco de pruebas [GLUE](https://gluebenchmark.com/). Contiene 67,349 reseñas de películas para el entrenamiento y 872 reseñas de películas para las pruebas. El conjunto de datos tiene dos clases: críticas de películas positivas y negativas.

In [ ]:
data_dir = tf.keras.utils.get_file(
      fname='SST-2.zip',
      origin='https://dl.fbaipublicfiles.com/glue/data/SST-2.zip',
      extract=True)
data_dir = os.path.join(os.path.dirname(data_dir), 'SST-2')

El conjunto de datos SST-2 se almacena en formato TSV. La única diferencia entre TSV y CSV es que TSV utiliza un tabulador `\t` como delimitador en lugar de una coma `,` en el formato CSV.

Éstas son las 5 primeras líneas del conjunto de datos de entrenamiento. label=0 significa negativo, label=1 significa positivo.

frase | etiqueta |  |  |
--- | --- | --- | --- | ---
ocultar nuevas secreciones de las unidades parentales | 0 |  |  |
no tiene ingenio, sólo gags trabajados | 0 |  |  |
que ama a sus personajes y comunica algo bastante hermoso sobre la naturaleza humana | 1 |  |  |
permanece totalmente satisfecho de seguir siendo el mismo a lo largo de | 0 |  |  |
con los peores clichés de La Venganza de los Nerds que los cineastas pudieron desenterrar | 0 |  |  |

Después, cargaremos el conjunto de datos en un marco de datos Pandas y cambiaremos los nombres de las etiquetas actuales (`0` y `1`) por otros más legibles para los humanos (`negative` y `positive`) y los usaremos para el entrenamiento del modelo.


In [ ]:
import pandas as pd

def replace_label(original_file, new_file):
  # Load the original file to pandas. We need to specify the separator as
  # '\t' as the training data is stored in TSV format
  df = pd.read_csv(original_file, sep='\t')

  # Define how we want to change the label name
  label_map = {0: 'negative', 1: 'positive'}

  # Excute the label change
  df.replace({'label': label_map}, inplace=True)

  # Write the updated dataset to a new file
  df.to_csv(new_file)

# Replace the label name for both the training and test dataset. Then write the
# updated CSV dataset to the current folder.
replace_label(os.path.join(os.path.join(data_dir, 'train.tsv')), 'train.csv')
replace_label(os.path.join(os.path.join(data_dir, 'dev.tsv')), 'dev.csv')

## Inicio rápido

El entrenamiento de un modelo de clasificación de textos consta de cinco pasos:

**Paso 1. Seleccione una arquitectura de modelo de clasificación de texto.**

Aquí usamos la arquitectura del modelo de incorporación de palabras promedio, que producirá un modelo pequeño y rápido con una precisión decente.

In [ ]:
spec = model_spec.get('average_word_vec')

Model Maker también admite otras arquitecturas de modelos como [BERT](https://arxiv.org/abs/1810.04805). Si está interesado en conocer otras arquitecturas, consulte la sección [Elija una arquitectura de modelo para el Clasificador de texto](#scrollTo=kJ_B8fMDOhMR) más abajo.

**Paso 2.   Cargue los datos de entrenamiento y de prueba y, a continuación, preprocesarlos de acuerdo con un `model_spec` específico.**

Model Maker puede tomar datos de entrada en formato CSV. Cargaremos el conjunto de datos de entrenamiento y de prueba con los nombres de etiquetas legibles por humanos que se crearon anteriormente.

Cada arquitectura de modelo requiere que los datos de entrada se procesen de una manera determinada. `DataLoader` lee el requisito desde `model_spec` y ejecuta automáticamente el preprocesamiento necesario.

In [ ]:
train_data = DataLoader.from_csv(
      filename='train.csv',
      text_column='sentence',
      label_column='label',
      model_spec=spec,
      is_training=True)
test_data = DataLoader.from_csv(
      filename='dev.csv',
      text_column='sentence',
      label_column='label',
      model_spec=spec,
      is_training=False)

**Paso 3. Entrene el modelo TensorFlow con los datos de entrenamiento.**

El modelo de incorporación de palabras promedio usa `batch_size = 32` de forma predeterminada. Por lo tanto, verá que se necesitan 2104 pasos para recorrer las 67,349 frases del conjunto de datos de entrenamiento. Entrenaremos el modelo durante 10 épocas, lo que significa recorrer el conjunto de datos de entrenamiento 10 veces.

In [ ]:
model = text_classifier.create(train_data, model_spec=spec, epochs=10)

**Paso 4. Evalúe el modelo con los datos de prueba.**

Después de entrenar el modelo de clasificación de texto usando las frases del conjunto de datos de entrenamiento, usaremos las 872 frases restantes del conjunto de datos de prueba para evaluar cómo se comporta el modelo frente a nuevos datos que nunca ha visto antes.

Como el tamaño predeterminado del lote es 32, se necesitarán 28 pasos para recorrer las 872 frases del conjunto de datos de prueba.

In [ ]:
loss, acc = model.evaluate(test_data)

**Paso 5. Exporte como modelo TensorFlow Lite.**

Vamos a exportar la clasificación de texto que hemos entrenado en el formato TensorFlow Lite. Especificaremos en qué carpeta exportar el modelo. De forma predeterminada, el modelo TFLite flotante se exporta para la arquitectura del modelo de incorporación de palabras Promedio.

In [ ]:
model.export(export_dir='average_word_vec')

Puede descargar el archivo del modelo de TensorFlow Lite usando la barra lateral izquierda de Colab. Vaya a la carpeta `average_word_vec` tal y como especificamos en el parámetro `export_dir` anterior, haga clic con el botón derecho del ratón en el archivo `model.tflite` y seleccione `Descargar` para descargarlo en su computadora local.

Este modelo puede integrarse en una app Android o iOS usando la [API NLClassifier](https://www.tensorflow.org/lite/inference_with_metadata/task_library/nl_classifier) de la [Biblioteca de tareas TensorFlow Lite](https://www.tensorflow.org/lite/inference_with_metadata/task_library/overview).

Consulte la app de ejemplo de [Clasificación de textos de TFLite](https://github.com/tensorflow/examples/blob/master/lite/examples/text_classification/android/lib_task_api/src/main/java/org/tensorflow/lite/examples/textclassification/client/TextClassificationClient.java#L54) para obtener más detalles sobre cómo se usa el modelo en una app operativa.

*Nota 1: Android Studio Model Binding aún no admite la clasificación de texto, por lo que deberá usar la biblioteca de tareas de TensorFlow Lite.*

*Nota 2: Existe un archivo `model.json` en la misma carpeta que el modelo TFLite. Contiene la representación JSON de los [metadatos](https://www.tensorflow.org/lite/models/convert/metadata) incluidos en el modelo de TensorFlow Lite. Los metadatos del modelo ayudan a la biblioteca de tareas de TFLite a saber qué hace el modelo y cómo preprocesar/postprocesar los datos para el modelo. No necesita descargar el archivo `model.json` ya que sólo tiene fines informativos y su contenido ya se encuentra dentro del archivo TFLite.*

*Nota 3: Si entrena un modelo de clasificación de texto utilizando la arquitectura MobileBERT o BERT-Base, tendrá que usar la [API BertNLClassifier](https://www.tensorflow.org/lite/inference_with_metadata/task_library/bert_nl_classifier) en su lugar para integrar el modelo entrenado en una app móvil.*

Las siguientes secciones recorren el ejemplo paso a paso para mostrar más detalles.

**Paso 6: Utilice la `biblioteca de tareas TFLite` para hacer una demo de cómo usar los modelos entrenados.**

Lea el archivo dev.csv en los datos de frases para predecir con el modelo entrenado

In [ ]:
sentence_data = pd.read_csv('/content/dev.csv', index_col=0)
sentence_data

Parámetro de configuración del modelo

In [ ]:
# Name of the TFLite text classification model.
_MODEL = '/content/average_word_vec/model.tflite'
# Whether to run the model on EdgeTPU.
_ENABLE_EDGETPU = False
# Number of CPU threads to run the model.
_NUM_THREADS = 4

Inicializar el modelo

También podemos cambiar los parámetros como `file_name`, `use_coral`, y `num_threads` que pueden afectar a los resultados del modelo. Los parámetros que puede modificar son:

- `file_name`: Nombre del modelo de clasificación de imágenes TFLite.
- `use_coral`: Si es verdadero, la inferencia se delegará a un dispositivo TPU Coral Edge conectado.
- `num_threads`: Número de hilos de CPU para ejecutar el modelo.

In [ ]:
# Initialize the text classification model.
base_options = core.BaseOptions(file_name=_MODEL, use_coral=_ENABLE_EDGETPU, num_threads=_NUM_THREADS)
options = text.NLClassifierOptions(base_options)

# Create NLClassifier from options.
classifier = text.NLClassifier.create_from_options(options)

Predecir usando la `biblioteca de tareas de TFLite`

In [ ]:
for idx in range(20):
  sentence = sentence_data['sentence'].iloc[idx]
  label = sentence_data['label'].iloc[idx]
  text_classification_result = classifier.classify(sentence)
  classification_list = text_classification_result.classifications[0].categories

  # Sort output by probability descending.
  predict_label = sorted(
      classification_list, key=lambda item: item.score, reverse=True)[0]

  print('truth_label: {} -----> predict_label: {}'.format(label, predict_label.category_name))

## Seleccionar una arquitectura de modelo para el clasificador de texto

Cada objeto `model_spec` representa un modelo específico para el clasificador de texto. Model Maker de TensorFlow Lite admite actualmente los modelos [MobileBERT](https://arxiv.org/pdf/2004.02984.pdf), la incorporación promedio de palabras y [BERT-Base](https://arxiv.org/pdf/1810.04805.pdf).

Modelos compatibles | Nombre de model_spec | Descripción del modelo | Tamaño del modelo
--- | --- | --- | ---
Incorporación promedio de palabras | 'average_word_vec' | Promediar las incorporaciones de palabras de texto con activación RELU. | &lt;1 MB
MobileBERT | 'mobilebert_classifier' | 4,3 veces más pequeño y 5.5 veces más rápido que el BERT-Base al tiempo que consigue resultados competitivos, adecuado para aplicaciones en dispositivos. | 25 MB con cuantización <br> 100 MB sin cuantización
BERT-Base | 'bert_classifier' | Proceso estándar BERT que se usa ampliamente en tareas de PNL. | 300 MB

En el inicio rápido, hemos usado el modelo de incorporación promedio de palabras. Cambiemos a [MobileBERT](https://arxiv.org/pdf/2004.02984.pdf) para entrenar un modelo con mayor precisión.

In [ ]:
mb_spec = model_spec.get('mobilebert_classifier')

## Cargar datos de entrenamiento

Puede subir su propio conjunto de datos para trabajar con este tutorial. Suba su conjunto de datos usando la barra lateral izquierda en Colab.

<img src="https://storage.googleapis.com/download.tensorflow.org/models/tflite/screenshots/model_maker_text_classification.png" width="800" hspace="100" alt="Subir archivo">

Si prefiere no subir su conjunto de datos a la nube, también puede ejecutar localmente la biblioteca siguiendo la [guía](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker).

Para no complicarnos, reutilizaremos el conjunto de datos SST-2 descargado anteriormente. Usemos el método `DataLoader.from_csv` para cargar los datos.

Tenga en cuenta que, como hemos cambiado la arquitectura del modelo, tendremos que volver a cargar el conjunto de datos de entrenamiento y de prueba para aplicar la nueva lógica de preprocesamiento.

In [ ]:
train_data = DataLoader.from_csv(
      filename='train.csv',
      text_column='sentence',
      label_column='label',
      model_spec=mb_spec,
      is_training=True)
test_data = DataLoader.from_csv(
      filename='dev.csv',
      text_column='sentence',
      label_column='label',
      model_spec=mb_spec,
      is_training=False)

La biblioteca Model Maker también admite el método `from_folder()` para cargar datos. Asume que los datos de texto de la misma clase se encuentran en el mismo subdirectorio y que el nombre de la subcarpeta es el nombre de la clase. Cada archivo de texto contiene una muestra de revisión de la película. El parámetro `class_labels` se usa para especificar cuáles son las subcarpetas.

## Entrenar un modelo TensorFlow

Entrene un modelo de clasificación de texto usando los datos de entrenamiento.

*Nota: Como MobileBERT es un modelo complejo, cada época de entrenamiento tardará unos 10 minutos en una GPU Colab. Asegúrese de usar un runtime de GPU.*

In [ ]:
model = text_classifier.create(train_data, model_spec=mb_spec, epochs=3)

Examine la estructura detallada del modelo.

In [ ]:
model.summary()

## Evaluar el modelo

Evalúe el modelo que acabamos de entrenar usando los datos de prueba y mida el valor de pérdida y precisión.

In [ ]:
loss, acc = model.evaluate(test_data)

## Exportar como modelo TensorFlow Lite

Convierta el modelo entrenado al formato de modelo TensorFlow Lite con [metadatos](https://www.tensorflow.org/lite/models/convert/metadata) para poder usarlo posteriormente en una aplicación ML en el dispositivo. El archivo de etiquetas y el archivo de vocabulario están incrustados en los metadatos. El nombre de archivo TFLite predeterminado es `model.tflite`.

En muchas aplicaciones de ML en el dispositivo, el tamaño del modelo es un factor importante. Por lo tanto, se recomienda aplicar la cuantización del modelo para hacerlo más pequeño y, potencialmente, ejecutarlo más rápido. La técnica de cuantización predeterminada tras el entrenamiento es la cuantización de rango dinámico para los modelos BERT y MobileBERT.

In [ ]:
model.export(export_dir='mobilebert/')

El archivo modelo de TensorFlow Lite puede integrarse en una app móvil usando la [API del BertNLClassifier](https://www.tensorflow.org/lite/inference_with_metadata/task_library/bert_nl_classifier) en la [Biblioteca de tareas de TensorFlow Lite](https://www.tensorflow.org/lite/inference_with_metadata/task_library/overview). Tenga en cuenta que ésta es **diferente** de la API `NLClassifier` utilizada para integrar la clasificación de texto entrenada con la arquitectura del modelo de vectores de palabras promedio.

Los formatos de exportación pueden ser uno o una lista de los siguientes:

- `ExportFormat.TFLITE`
- `ExportFormat.LABEL`
- `ExportFormat.VOCAB`
- `ExportFormat.SAVED_MODEL`

De forma predeterminada, sólo exporta el archivo del modelo TensorFlow Lite que contiene los metadatos del modelo. También puede seleccionar exportar otros archivos relacionados con el modelo para examinarlo mejor.Por ejemplo, exportando sólo el archivo de etiquetas y el archivo de vocabulario de la siguiente manera:

In [ ]:
model.export(export_dir='mobilebert/', export_format=[ExportFormat.LABEL, ExportFormat.VOCAB])

Puede evaluar el modelo TFLite con el método `evaluate_tflite` para medir su precisión. Si convierte el modelo TensorFlow entrenado al formato TFLite y aplica la cuantización puede afectar su precisión, por lo que se recomienda evaluar la precisión del modelo TFLite antes de implementarlo.

In [ ]:
accuracy = model.evaluate_tflite('mobilebert/model.tflite', test_data)
print('TFLite model accuracy: ', accuracy)

## Uso avanzado

La función `create` es la función de controlador que usa la biblioteca Model Maker para crear modelos. El parámetro `model_spec` define la especificación del modelo. Actualmente se admiten las clases `AverageWordVecSpec` y `BertClassifierSpec`. La función `create` consta de los siguientes pasos:

1. Crea el modelo para el clasificador de texto según `model_spec`.
2. Entrena el modelo clasificador. Las épocas predeterminadas y el tamaño predeterminado del lote se configuran mediante las variables `default_training_epochs` y `default_batch_size` del objeto `model_spec`.

Esta sección cubre temas de uso avanzado como el ajuste del modelo y los hiperparámetros de entrenamiento.

### Personalizar los hiperparámetros del modelo MobileBERT

Los parámetros del modelo que puede ajustar son:

- `seq_len`: Longitud de la secuencia a introducir en el modelo.
- `initializer_range`: La desviación estándar del `truncated_normal_initializer` para inicializar todas las matrices de ponderación.``
- `trainable`: Booleano que especifica si la capa preentrenada es entrenable.

Los parámetros del canal de entrenamiento que puede ajustar son:

- `model_dir`: La ubicación de los archivos de puntos de verificación del modelo. Si no está contigurada, se usará un directorio temporal.
- `dropout_rate`: La tasa de abandono.
- `learning_rate`: La tasa de aprendizaje inicial para el optimizador Adam.
- `tpu`: Dirección TPU a la que conectarse.

Por ejemplo, puede ajustar el `seq_len=256` (de forma predeterminada es 128). Esto permite al modelo clasificar textos más largos.

In [ ]:
new_model_spec = model_spec.get('mobilebert_classifier')
new_model_spec.seq_len = 256

### Personalizar los hiperparámetros del modelo de incorporación de palabras Promedio

Puede ajustar la infraestructura del modelo como las variables `wordvec_dim` y `seq_len` de la clase `AverageWordVecSpec`.


Por ejemplo, puede entrenar el modelo con un valor mayor de `wordvec_dim`. Tenga en cuenta que debe construir un nuevo `model_spec` si modifica el modelo.

In [ ]:
new_model_spec = AverageWordVecSpec(wordvec_dim=32)

Obtenga los datos preprocesados.

In [ ]:
new_train_data = DataLoader.from_csv(
      filename='train.csv',
      text_column='sentence',
      label_column='label',
      model_spec=new_model_spec,
      is_training=True)

Entrene el nuevo modelo.

In [ ]:
model = text_classifier.create(new_train_data, model_spec=new_model_spec)

### Afinar los hiperparámetros de entrenamiento

También puede ajustar los hiperparámetros de entrenamiento como `epochs` y `batch_size` para influir en el rendimiento del modelo. Por ejemplo:

- `epochs`: Un mayor número de épocas podría lograr una mayor precisión, pero podría provocar un ajuste excesivo.
- `batch_size`: el número de muestras a usar en un paso de entrenamiento.

Por ejemplo, puede entrenarse con más épocas.

In [ ]:
model = text_classifier.create(new_train_data, model_spec=new_model_spec, epochs=20)

Evalúe el modelo recién reentrenado con 20 épocas de entrenamiento.

In [ ]:
new_test_data = DataLoader.from_csv(
      filename='dev.csv',
      text_column='sentence',
      label_column='label',
      model_spec=new_model_spec,
      is_training=False)

loss, accuracy = model.evaluate(new_test_data)

### Cambiar la arquitectura del modelo

Puede cambiar el modelo modificando el `model_spec`. A continuación se muestra cómo cambiar al modelo BERT-Base.

Cambie el `model_spec` al modelo BERT-Base para el clasificador de texto.

In [ ]:
spec = model_spec.get('bert_classifier')

Los pasos restantes son los mismos.

### Personalizar la cuantización posterior al entrenamiento en el modelo TensorFlow Lite

[La cuantización posterior al entrenamiento](https://www.tensorflow.org/lite/performance/post_training_quantization) es una técnica de conversión que puede reducir el tamaño del modelo y la latencia de la inferencia, al tiempo que mejora la velocidad de inferencia de la CPU y del acelerador de hardware, con una pequeña degradación de la precisión del modelo. Por ello, se usa mucho para optimizar el modelo.

La biblioteca Model Maker aplica una técnica predeterminada de cuantización postentrenamiento al exportar el modelo. Si desea personalizar la cuantización postentrenamiento, Model Maker también soporta múltiples opciones de cuantización postentrenamiento utilizando [QuantizationConfig](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/config/QuantizationConfig). Tomemos como ejemplo la cuantización float16. En primer lugar, defina la configuración de cuantificación.

```python
config = QuantizationConfig.for_float16()
```

A continuación, exportamos el modelo TensorFlow Lite con dicha configuración.

```python
model.export(export_dir='.', tflite_filename='model_fp16.tflite', quantization_config=config)
```

# Más información

Puede leer nuestro ejemplo de [clasificación de texto](https://www.tensorflow.org/lite/examples/text_classification/overview) para conocer los detalles técnicos. Para más información, consulte:

- [Guía](https://www.tensorflow.org/lite/models/modify/model_maker) y [Referencia de API](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker) de Model Maker de TensorFlow Lite.
- Biblioteca de tareas: [NLClassifier](https://www.tensorflow.org/lite/inference_with_metadata/task_library/nl_classifier) y [BertNLClassifier](https://www.tensorflow.org/lite/inference_with_metadata/task_library/bert_nl_classifier) para su implementación.
- Las apps de referencia de principio a fin: [Android](https://github.com/tensorflow/examples/tree/master/lite/examples/text_classification/android) y [iOS](https://github.com/tensorflow/examples/tree/master/lite/examples/text_classification/ios).